# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), May 15, 2021**

In [1]:
using DataFrames

In [2]:
using BenchmarkTools

In [3]:
using CategoricalArrays

In [4]:
using PooledArrays

## Performance tips

### Access by column number is faster than by name

In [5]:
x = DataFrame(rand(5, 1000), :auto)
@btime $x[!, 500];
@btime $x.x500;

  5.832 ns (0 allocations: 0 bytes)
  24.397 ns (1 allocation: 32 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [6]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  78.966 ms (5999015 allocations: 122.06 MiB)


In [7]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  5.644 ms (35 allocations: 30.52 MiB)
  5.596 ms (35 allocations: 30.52 MiB)


In [8]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  5.451 ms (35 allocations: 30.52 MiB)


In general for tall and narrow tables it is often useful to use `Tables.rowtable`, `Tables.columntable` or `Tables.namedtupleiterator` for intermediate processing of data in a type-stable way.

### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between copying vs non-copying data frame creation

In [9]:
function f1()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto, copycols=false) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto, copycols=false) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  24.082 ms (1949523 allocations: 37.40 MiB)
  26.004 ms (1949723 allocations: 45.04 MiB)
  3.932 ms (623 allocations: 7.66 MiB)
  5.144 ms (823 allocations: 15.30 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [10]:
x = DataFrame(rand(10^6, 5), :auto)
y = DataFrame(transpose(1.0:5.0), :auto)
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5), :auto) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  3.413 ms (153 allocations: 38.16 MiB)
  999.200 ns (17 allocations: 832 bytes)
  406.851 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [11]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  3.505 ms (7 allocations: 7.63 MiB)
 categorical:
  14.791 ms (1000004 allocations: 30.52 MiB)
String
 raw:
  24.178 ms (4 allocations: 608 bytes)
 categorical:
  27.281 ms (1000004 allocations: 30.52 MiB)
Union{Missing, Int64}
 raw:
  5.706 ms (4 allocations: 624 bytes)
 categorical:
  15.537 ms (1000004 allocations: 30.52 MiB)
Union{Missing, String}
 raw:
  14.729 ms (4 allocations: 608 bytes)
 categorical:
  22.769 ms (1000004 allocations: 30.52 MiB)


### When aggregating use column selector and prefer categorical or pooled array grouping variable

In [12]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [13]:
gdf = groupby(df, :x)

,x,y
,Char,Int64
1,b,1
2,b,1
3,b,1
4,b,1
5,b,1
6,b,1
7,b,1
8,b,1
9,b,1


In [14]:
@btime combine(v -> sum(v.y), $gdf) # traditional syntax, slow

  55.567 ms (299 allocations: 19.08 MiB)


,x,x1
,Char,Int64
1,b,2499032
2,c,2500708
3,d,2499443
4,a,2500817


In [15]:
@btime combine($gdf, :y=>sum) # use column selector

  12.366 ms (204 allocations: 14.47 KiB)


,x,y_sum
,Char,Int64
1,b,2499032
2,c,2500708
3,d,2499443
4,a,2500817


In [16]:
transform!(df, :x => categorical => :x);

In [17]:
gdf = groupby(df, :x)

,x,y
,Cat…,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [18]:
@btime combine($gdf, :y=>sum)

  12.659 ms (211 allocations: 15.12 KiB)


,x,y_sum
,Cat…,Int64
1,a,2500817
2,b,2499032
3,c,2500708
4,d,2499443


In [19]:
transform!(df, :x => PooledArray{Char} => :x)

,x,y
,Char,Int64
1,b,1
2,c,1
3,d,1
4,d,1
5,d,1
6,a,1
7,a,1
8,d,1
9,b,1


In [20]:
gdf = groupby(df, :x)

,x,y
,Char,Int64
1,b,1
2,b,1
3,b,1
4,b,1
5,b,1
6,b,1
7,b,1
8,b,1
9,b,1


In [21]:
@btime combine($gdf, :y=>sum)

  12.579 ms (206 allocations: 14.53 KiB)


,x,y_sum
,Char,Int64
1,b,2499032
2,c,2500708
3,d,2499443
4,a,2500817


### Use views instead of materializing a new DataFrame

In [22]:
x = DataFrame(rand(100, 1000), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208
2,0.259236,0.689041,0.653378,0.290464,0.798324,0.679658,0.992848,0.549183
3,0.662806,0.661069,0.414882,0.0283973,0.0725591,0.197549,0.479039,0.0510472
4,0.0950638,0.299294,0.68422,0.0325769,0.285205,0.274216,0.148808,0.887337
5,0.49026,0.887202,0.944367,0.346795,0.984099,0.273077,0.099737,0.690124
6,0.406854,0.60399,0.333629,0.0830925,0.157933,0.541711,0.48406,0.862885
7,0.499201,0.581658,0.366791,0.904021,0.527428,0.322,0.726671,0.0845636
8,0.158473,0.502546,0.992313,0.300413,0.731632,0.765302,0.857465,0.41926
9,0.971737,0.28474,0.513548,0.369635,0.186662,0.537873,0.296569,0.57022


In [23]:
@btime $x[1:1, :]

  169.615 μs (1985 allocations: 159.44 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208


In [24]:
@btime $x[1, :]

  21.486 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208


In [25]:
@btime view($x, 1:1, :)

  21.966 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208


In [26]:
@btime $x[1:1, 1:20]

  3.635 μs (30 allocations: 3.53 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208


In [27]:
@btime $x[1, 1:20]

  19.697 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208


In [28]:
@btime view($x, 1:1, 1:20)

  19.713 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.766316,0.316707,0.343028,0.965914,0.62647,0.0856604,0.904061,0.321208
